In [255]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from CardioMesh.CardiacMesh import Cardiac3DMesh

In [256]:
ID = "1000215"

fhm_mesh = Cardiac3DMesh(
    filename=f"/home/rodrigo/01_repos/CardiacCOMA/data/cardio/meshes/by_id/{ID}/models/FHM_time001.npy",
    faces_filename="/home/rodrigo/01_repos/CardioMesh/data/faces_fhm_10pct_decimation.csv",
    subpart_id_filename="/home/rodrigo/01_repos/CardioMesh/data/subpartIDs_FHM_10pct.txt"
)

lv_subparts = ("LV")
lv_mesh = fhm_mesh["LV"]

### Remove base region from LV mesh

Compute principal axes

In [222]:
# Compute the mean of the point cloud
point_cloud = lv_mesh.v
mean = np.mean(point_cloud, axis=0)

# Subtract the mean from the point cloud to center it
centered_point_cloud = point_cloud - mean

# Compute the covariance matrix
covariance_matrix = np.cov(centered_point_cloud, rowvar=False)

# Compute the eigenvalues and eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

# Sort the eigenvalues and eigenvectors in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# The eigenvectors represent the principal inertia axes
principal_axes = eigenvectors
### Compute principal inertia axes
v_axis = principal_axes[:, 0] #
h_axis_1 = principal_axes[:, 1] # aligned with aorta
h_axis_2 = principal_axes[:, 2]

####################################################################################################
# HORTIZONTAL PLANES
# Determine the position of the slicing planes along this axis
min_pos = np.min(centered_point_cloud.dot(v_axis))
max_pos = np.max(centered_point_cloud.dot(v_axis))
# slice_thickness = (max_pos - min_pos) / (num_slices-1)
fractions = [ -0.01, 0.75, 1.01 ]
num_slices = len(fractions) - 1  
slice_positions = [ min_pos+fraction*(max_pos-min_pos) for fraction in fractions ]

condition_slice = {}
for j in range(num_slices):
    # Classify points based on which side of the slicing plane they fall
    above_plane = centered_point_cloud.dot(v_axis) > slice_positions[j]
    below_plane = centered_point_cloud.dot(v_axis) < slice_positions[j+1]
    condition_slice[j] = above_plane & below_plane
    
print(condition_slice[0].sum())
print(condition_slice[1].sum())

2758
1638


In [232]:
all_indices = np.array(range(len(lv_mesh.v)))
indices_without_top = set(all_indices[condition_slice[0]])
vertex_mapping = { j: i for i, j in enumerate(all_indices[condition_slice[0]]) }
vertex_reverse_mapping = { i: j for i, j in enumerate(all_indices[condition_slice[0]]) }

Create a mesh that keeps only the vertices not belong to the base, and computes a new mesh connectivity

In [233]:
lv_without_top = copy.deepcopy(lv_mesh)

# all_indices = np.array(range(len(lv_without_top.v)))
indices_without_top = set(all_indices[condition_slice[0]])

keep_face = np.zeros(len(lv_without_top.f)).astype(bool)

for i, face in enumerate(lv_without_top.f):
    keep_face[i] = all([vert in indices_without_top for vert in face])
    
lv_without_top.points = lv_without_top.points[condition_slice[0]]
lv_without_top.triangles = lv_without_top.triangles[keep_face]

for i, triangle in enumerate(lv_without_top.triangles):
    
    lv_without_top.triangles[i] = [ vertex_mapping[vert] for vert in triangle ]
    # print(lv_without_top.triangles[i])

Compute new edges and adjacency matrix, based on the triangles

In [234]:
from scipy import sparse as sp

lv_without_top._edges = lv_without_top._edges_from_triangles(lv_without_top.triangles)

lv_without_top._adj_matrix = sp.csc_matrix((
    np.ones(len(lv_without_top._edges)),
    ([x[0] for x in lv_without_top._edges], [x[1] for x in lv_without_top._edges]),
))

lv_without_top._adj_matrix

<2758x2758 sparse matrix of type '<class 'numpy.float64'>'
	with 16256 stored elements in Compressed Sparse Column format>

Compute dictionary of neighbors-per-vertex

In [235]:
neighbors_dict = {}
all_indices = all_indices = np.array(range(len(lv_without_top.v)))

for i, _ in enumerate(lv_without_top.v):
    neighbors_dict[i] = all_indices[np.array((lv_without_top.adj_matrix[:,i] > 0).todense()).flatten()]    

Find connected components to separate epi from endo:

In [236]:
def find_connected_components(graph):
    visited = set()
    components = []

    def dfs(node, component):
        visited.add(node)
        component.append(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, component)

    for node in graph:
        if node not in visited:
            component = []
            dfs(node, component)
            components.append(component)

    return components

# Usage
components = find_connected_components(neighbors_dict)

for component in components:
    print(len(component))

1446
1311
1


In [237]:
# epi_endo_labels = np.zeros(len(lv_without_top.points))
# epi_endo_labels[components[0]] = 100
# epi_endo_labels[components[1]] = 200
# 
# import meshio
# meshio.write_points_cells(
#     "lv_mesh_with_epi_endo.vtk",
#     lv_without_top.points,
#     cells={"triangle": np.array(lv_without_top.f)},
#     point_data={"subpartID": epi_endo_labels},
# )

Map vertices in the subsetted mesh to vertex's ids from the original mesh

In [242]:
epi = [vertex_reverse_mapping[v] for v in components[0]]
endo = [vertex_reverse_mapping[v] for v in components[1]]

In [253]:
epi_endo_labels = np.zeros(len(lv_mesh.points))
epi_endo_labels[epi] = 1
epi_endo_labels[endo] = 2

In [254]:
meshio.write_points_cells(
    "lv_mesh_with_epi_endo.vtk",
    lv_mesh.points,
    cells={"triangle": np.array(lv_mesh.f)},
    point_data={"subpartID": epi_endo_labels},
)